<a href="https://colab.research.google.com/github/humayraR/LeetspeakMining/blob/main/Another_copy_of_translateTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
from tqdm import tqdm, trange
import numpy as np
import json

In [2]:
! git clone https://github.com/humayraR/LeetspeakMining.git

import sys
sys.path.insert(0,'/content/LeetspeakMining')

%cd LeetspeakMining/youtube

fatal: destination path 'LeetspeakMining' already exists and is not an empty directory.
/content/LeetspeakMining/youtube


In [3]:
# load test data

test = pd.read_csv("test_yt.csv")
test.info()
print("Test Dataset:")
test

#test["reply"].values.size

#test["keyword"].values[0] -> 'weather'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57645 entries, 0 to 57644
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    57645 non-null  object
dtypes: object(1)
memory usage: 450.5+ KB
Test Dataset:


,text
0,Ok I hate to tell everyone but 90s quality is ...
1,Actually the recent windows update made my pc ...
2,Tom I really hope you enjoyed your stay in War...
3,"OR, more likely, they hired a professional who..."
4,I am confused because of Linus's tshirt.
...,...
57640,I think I remember Rome testing its water with...
57641,"""<a href=""""https://www.youtube.com/watch?v=G02..."
57642,Lasers revert Tom Scott brain back to a 12 yea...
57643,I don't care what religion you follow. If you ...


In [4]:
#setup pre-trained roberta model from https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
# Run for Roberta Model
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

print(polarity_scores_roberta("I am happy"))

{'roberta_neg': 0.007794359, 'roberta_neu': 0.030709105, 'roberta_pos': 0.96149653}


# LeetSpeak processing

In [6]:
! git clone https://github.com/humayraR/LeetspeakMining.git

import sys
sys.path.insert(0,'/content/LeetspeakMining')

%cd LeetspeakMining/youtube

fatal: destination path 'LeetspeakMining' already exists and is not an empty directory.
/content/LeetspeakMining/youtube/LeetspeakMining/youtube


In [9]:
import leetTranslate as lt
test2_result = {
    #'keyword': [],
    'text': [],
    'TranslatedReply': [],
    'Neg': [],
    'Neu': [],
    'Pos': []
}

# Assuming test is a DataFrame or a dictionary-like object that contains 'reply' and 'keyword'.
for i in trange(len(test['text'].values)):
    original_reply = test['text'].values[i]
    #keyword = test['keyword'].values[i]

    # Translate leetspeak back to English word by word using your leetTranslate function

    translated_words = [lt.caller(word) for word in original_reply.split()]
    translated_reply = ' '.join(translated_words)

    # Sentiment analysis on the translated English sentence
    english_sentiment_scores = polarity_scores_roberta(translated_reply)

    # Append results to the dictionary
    #test2_result['keyword'].append(keyword)
    test2_result['text'].append(original_reply)
    test2_result['TranslatedReply'].append(translated_reply)
    test2_result['Neg'].append(english_sentiment_scores['roberta_neg'])
    test2_result['Neu'].append(english_sentiment_scores['roberta_neu'])
    test2_result['Pos'].append(english_sentiment_scores['roberta_pos'])

# Convert the results dictionary to a DataFrame
test2_result_df = pd.DataFrame(test2_result)
print(test2_result_df.head())

 15%|█▌        | 8900/57645 [9:20:07<51:07:46,  3.78s/it]


KeyboardInterrupt: ignored

In [10]:
trans_result_df = pd.DataFrame(test2_result)
print(test2_result_df.head())

                                                text  \
0  Ok I hate to tell everyone but 90s quality is ...   

                                     TranslatedReply       Neg       Neu  \
0  Ok I hate to tell everyone but gos quality is ...  0.158345  0.251606   

        Pos  
0  0.590049  


In [11]:
trans_result_df.to_csv('test2_result_yt.csv', index=False)